В команде мы построили работу следующим образом: решили взять разные моделька каждый свою и выжать из нее максимум для задачи

Моя моделька: Detectron2

In [1]:
!pip install git+https://github.com/facebookresearch/detectron2.git@v0.6

  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-req-build-tmnsv1g9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-tmnsv1g9
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.9 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6418104 sha256=ff6daa987a3c9876778cdc2bcd52a2948a7c1690baeabf16c498848042d

In [7]:
!pip uninstall -y Pillow
!pip install Pillow==8.4.0
import site
import sys
from importlib import reload
reload(site)

  Using cached Pillow-8.4.0.tar.gz (49.4 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-8.4.0-cp311-cp311-linux_x86_64.whl size=1171652 sha256=151097aa6acdc99dcdfa014f3a65eeda1264a593844df3e04fc444d8b13aec75
  Stored in directory: /root/.cache/pip/wheels/c7/cf/6a/49767496ca114325703308efc0519f97126e3ca06dd91845b7
Successfully built Pillow
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires pillow>=10.1, but you have pillow 8.4.0 which is incompatible.
torchtune 0.6.1 requires Pillow>=9.4.0, but you have pillow 8.4.0 which is incompatible.
fastai 2.7.19 requires pillow>=9.0.0, but you have pillow 8.4.0 which is incompatible.


<module 'site' (frozen)>

In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

print(f"Detectron2 version: {detectron2.__version__}")
print("Установка прошла успешно!")

Detectron2 version: 0.6
Установка прошла успешно!


In [12]:
import torch
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper
import numpy as np
import os
import cv2

In [4]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import cv2
import os

classes = ["damage", "corrosion", "lightning", "lightning receptor", "missing teeth", "patch"]

for name in ["train_dataset", "val_dataset"]:
    if name in DatasetCatalog.list():
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)

register_coco_instances("train_dataset", {}, "/content/drive/MyDrive/Wind Turbine damage/train/_annotations.coco.json", "/content/Wind Turbine damage/train/")
register_coco_instances("val_dataset", {}, "/content/drive/MyDrive/Wind Turbine damage/valid/_annotations.coco.json", "/content/Wind Turbine damage/valid/")

MetadataCatalog.get("train_dataset").thing_classes = classes
MetadataCatalog.get("val_dataset").thing_classes = classes

dataset_name = "train_dataset"
dataset_dicts = DatasetCatalog.get(dataset_name)
metadata = MetadataCatalog.get(dataset_name)

num_images_to_visualize = 5

for d in random.sample(dataset_dicts, num_images_to_visualize):
    img = cv2.imread(d["file_name"])
    if img is None:
        print(f"Ошибка: Не удалось загрузить изображение {d['file_name']}")
        continue

[06/03 18:43:37 d2.data.datasets.coco]: Loading /content/drive/MyDrive/Wind Turbine damage/train/_annotations.coco.json takes 4.31 seconds.
WARNING [06/03 18:43:37 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/03 18:43:37 d2.data.datasets.coco]: Loaded 1070 images in COCO format from /content/drive/MyDrive/Wind Turbine damage/train/_annotations.coco.json
Ошибка: Не удалось загрузить изображение /content/Wind Turbine damage/train/DJI_0029_JPG.rf.c868e504f0d460ff990dae886f6a551d.jpg
Ошибка: Не удалось загрузить изображение /content/Wind Turbine damage/train/20240120_163647_jpg.rf.d609884c32b1a5637fd12c9db78760ff.jpg
Ошибка: Не удалось загрузить изображение /content/Wind Turbine damage/train/20240120_155211_mp4-0092_jpg.rf.3d902d08081341e1ad3abb7081d9ccbd.jpg
Ошибка: Не удалось загрузить изображение /content/Wind Turbine damage/train/DJI_0598-1-_JPG.rf.daa4e2b20acb1f41b2677250c18d6f4e.jpg
Ошибка: Не удалось загрузить 

In [15]:
classes = ["damage","corrosion", "lightning", "lightning receptor", "missing teeth", "patch"]

for name in ["train_dataset", "val_dataset"]:
    if name in DatasetCatalog.list():
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)

train_json_path = "/content/drive/MyDrive/Wind Turbine damage/train/_annotations.coco.json"
train_image_dir = "/content/drive/MyDrive/Wind Turbine damage/train/"
val_json_path = "/content/drive/MyDrive/Wind Turbine damage/valid/_annotations.coco.json"
val_image_dir = "/content/drive/MyDrive/Wind Turbine damage/valid/"

if not os.path.exists(train_json_path):
    print(f"Ошибка: Файл аннотаций для обучения не найден: {train_json_path}")

if not os.path.exists(val_json_path):
    print(f"Ошибка: Файл аннотаций для валидации не найден: {val_json_path}")

register_coco_instances("train_dataset", {}, train_json_path, train_image_dir)
register_coco_instances("val_dataset", {}, val_json_path, val_image_dir)

MetadataCatalog.get("train_dataset").thing_classes = classes
MetadataCatalog.get("val_dataset").thing_classes = classes

In [16]:
class CustomDatasetMapper(DatasetMapper):
    def __init__(self, cfg, is_train=True):
        super().__init__(cfg, is_train)
        self.augmentations = [
            T.ResizeShortestEdge(
                cfg.INPUT.MIN_SIZE_TRAIN,
                cfg.INPUT.MAX_SIZE_TRAIN,
                cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING
            ),
            T.RandomFlip(horizontal=True)
        ]
        if is_train:
            self.augmentations.extend([
            ])

    def __call__(self, dataset_dict):
        dataset_dict = dataset_dict.copy()
        image = utils.read_image(dataset_dict["file_name"], format=self.image_format)

        aug_input = T.AugInput(image)
        transforms = T.AugmentationList(self.augmentations)(aug_input)
        image = aug_input.image

        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
        ]
        valid_annos = []
        for anno in annos:
            x1, y1, w, h = anno["bbox"]
            if w > 0 and h > 0 and x1 >= 0 and y1 >= 0 and x1 + w <= image.shape[1] and y1 + h <= image.shape[0]:
                valid_annos.append(anno)

        return {
            "image": torch.as_tensor(image.transpose(2, 0, 1).astype("float32")),
            "instances": utils.annotations_to_instances(valid_annos, image.shape[:2]),
            "height": image.shape[0],
            "width": image.shape[1],
        }

In [21]:
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=CustomDatasetMapper(cfg, is_train=True))

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, output_dir=output_folder)

# --- 5. Конфигурация модели и параметров обучения ---
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {cfg.MODEL.DEVICE}")

# Датасеты для обучения и тестирования
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ("val_dataset",) # Используем наш валидационный датасет для оценки
cfg.DATALOADER.NUM_WORKERS = 2 # 2-4 обычно хорошее значение
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice" # Случайный выбор из MIN_SIZE_TRAIN

cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TEST = 1333


cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)


cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.STEPS = (10000, 13000)
cfg.SOLVER.GAMMA = 0.1
cfg.TEST.EVAL_PERIOD = 1000


Используемое устройство: cuda


In [22]:
cfg.OUTPUT_DIR = "./output_wind_turbine_damage"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

print("Начало обучения...")
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
print("Обучение завершено.")

Начало обучения...
[06/03 20:33:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[06/03 20:33:43 d2.engine.train_loop]: Starting training from iteration 0
[06/03 20:34:25 d2.utils.events]:  eta: 0:41:36  iter: 19  total_loss: 5.041  loss_cls: 2.019  loss_box_reg: 0.000435  loss_rpn_cls: 2.511  loss_rpn_loc: 0.4273  time: 1.9269  data_time: 0.9743  lr: 4.9953e-06  max_mem: 5030M
[06/03 20:35:05 d2.utils.events]:  eta: 0:41:02  iter: 39  total_loss: 4.229  loss_cls: 1.784  loss_box_reg: 0.003677  loss_rpn_cls: 1.933  loss_rpn_loc: 0.4374  time: 1.9678  data_time: 1.0028  lr: 9.9902e-06  max_mem: 5030M
[06/03 20:35:47 d2.utils.events]:  eta: 0:42:55  iter: 59  total_loss: 2.491  loss_cls: 1.359  loss_box_reg: 0.002158  loss_rpn_cls: 0.8653  loss_rpn_loc: 0.4063  time: 2.0110  data_time: 1.0262  lr: 1.4985e-05  max_mem: 5030M


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (149817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[06/03 20:36:33 d2.utils.events]:  eta: 0:42:45  iter: 79  total_loss: 1.401  loss_cls: 0.846  loss_box_reg: 0.001746  loss_rpn_cls: 0.3404  loss_rpn_loc: 0.1817  time: 2.0911  data_time: 1.3119  lr: 1.998e-05  max_mem: 5030M
[06/03 20:37:10 d2.utils.events]:  eta: 0:42:47  iter: 99  total_loss: 0.7332  loss_cls: 0.2719  loss_box_reg: 0.003132  loss_rpn_cls: 0.137  loss_rpn_loc: 0.2872  time: 2.0402  data_time: 0.8880  lr: 2.4975e-05  max_mem: 5030M
[06/03 20:37:52 d2.utils.events]:  eta: 0:41:33  iter: 119  total_loss: 0.5805  loss_cls: 0.1309  loss_box_reg: 0.001102  loss_rpn_cls: 0.1532  loss_rpn_loc: 0.2418  time: 2.0507  data_time: 1.1128  lr: 2.997e-05  max_mem: 5030M
[06/03 20:38:34 d2.utils.events]:  eta: 0:40:56  iter: 139  total_loss: 0.4809  loss_cls: 0.08598  loss_box_reg: 0.004648  loss_rpn_cls: 0.1534  loss_rpn_loc: 0.2437  time: 2.0600  data_time: 1.1212  lr: 3.4965e-05  max_mem: 5030M


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (149817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[06/03 20:39:17 d2.utils.events]:  eta: 0:40:20  iter: 159  total_loss: 0.4802  loss_cls: 0.09008  loss_box_reg: 0.005441  loss_rpn_cls: 0.1219  loss_rpn_loc: 0.2428  time: 2.0666  data_time: 1.1235  lr: 3.996e-05  max_mem: 5030M
[06/03 20:39:55 d2.utils.events]:  eta: 0:39:45  iter: 179  total_loss: 0.4387  loss_cls: 0.07633  loss_box_reg: 0.01148  loss_rpn_cls: 0.1209  loss_rpn_loc: 0.2168  time: 2.0524  data_time: 0.9087  lr: 4.4955e-05  max_mem: 5030M
[06/03 20:40:35 d2.utils.events]:  eta: 0:39:44  iter: 199  total_loss: 0.4236  loss_cls: 0.05699  loss_box_reg: 0.006574  loss_rpn_cls: 0.1384  loss_rpn_loc: 0.2012  time: 2.0427  data_time: 1.0197  lr: 4.995e-05  max_mem: 5030M
[06/03 20:41:16 d2.utils.events]:  eta: 0:39:33  iter: 219  total_loss: 0.5433  loss_cls: 0.09914  loss_box_reg: 0.01793  loss_rpn_cls: 0.113  loss_rpn_loc: 0.2684  time: 2.0439  data_time: 1.0366  lr: 5.4945e-05  max_mem: 5030M
[06/03 20:41:55 d2.utils.events]:  eta: 0:39:30  iter: 239  total_loss: 0.4462  l

In [23]:
import os
import csv
import cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import detection_utils as utils
from detectron2 import model_zoo

cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

# Создаем предиктор.
predictor = DefaultPredictor(cfg)

test_dataset_dicts = DatasetCatalog.get("test_dataset")
test_metadata = MetadataCatalog.get("test_dataset")

predictions_csv_path = "predictions.csv"
with open(predictions_csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image_id", "objects"])


    for i, d in enumerate(test_dataset_dicts):
        if i % 100 == 0:
            print(f"Processing image {i+1}/{len(test_dataset_dicts)}")
        im = utils.read_image(d["file_name"], format="BGR")
        outputs = predictor(im)

        instances = outputs["instances"].to("cpu")
        boxes = instances.pred_boxes.tensor.numpy()
        scores = instances.scores.numpy()
        pred_classes = instances.pred_classes.numpy()

        objects_output_format = []
        for box, score, cls_id in zip(boxes, scores, pred_classes):
            x1, y1, x2, y2 = box
            w = x2 - x1
            h = y2 - y1
            objects_output_format.append([
                test_metadata.thing_classes[cls_id],
                float(x1),
                float(y1),
                float(w),
                float(h),
                float(score)
            ])

        image_id = os.path.basename(d["file_name"])

        objects_string_for_csv = str(objects_output_format).replace("'", '"')
        writer.writerow([image_id, objects_string_for_csv])

print(f"Inference finished. Predictions saved to: {predictions_csv_path}")
print(f"Example output format in CSV for the last processed image (image_id, objects):")
print(f"{image_id},{objects_string_for_csv}")

WARNING [06/03 21:28:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/03 21:28:56 d2.data.datasets.coco]: Loaded 271 images in COCO format from /content/drive/MyDrive/Wind Turbine damage/valid/_annotations.coco.json
Processing image 1/271
Processing image 101/271
Processing image 201/271
Inference finished. Predictions saved to: predictions.csv
Example output format in CSV for the last processed image (image_id, objects):
20240120_155211_mp4-0012_jpg.rf.1b3bd46c3fee8605cb7a8e69f61c4fde.jpg,[]
